In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forest_risks import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forest_risks.utils import get_store
import altair as alt
from carbonplan.data import cat
import rioxarray
import cartopy.crs as ccrs
import cartopy
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')
import matplotlib
from carbonplan_data import utils

alt.themes.enable("carbonplan_light")

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
coarsen = 4
mask = (
    (load.nlcd(store="az", year=2001).sel(band=[41, 42, 43, 90]).sum("band") > 0.25)
    .astype("float")
    .coarsen(x=coarsen, y=coarsen, boundary="trim")
    .mean()
)

### load in fire data


In [ ]:
historical_fire = xr.open_zarr(
    "https://carbonplan.blob.core.windows.net/carbonplan-forests/risks/results/paper/fire_terraclimate.zarr"
).load()
fire_mask = ~np.isnan(historical_fire.historical.isel(time=0).drop("time"))

In [ ]:
ds = (
    xr.open_zarr(
        "https://carbonplan.blob.core.windows.net/carbonplan-forests/risks/results/paper/fire_cmip.zarr"
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .where(fire_mask)
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .compute()
) * 100  # scale to percent

In [ ]:
maps = {}
maps["Fire"] = (
    ds.sel(scenario="ssp370", year=slice("2080", "2099"))
    .mean(dim="year")
    .mean(dim="gcm")
    .drop("scenario")
    .compute()
    .probability
)

### load insects and drought


In [ ]:
for variable in ["drought", "insects"]:
    maps[variable.capitalize()] = (
        xr.open_zarr(
            "https://carbonplan.blob.core.windows.net/carbonplan-forests/risks/results/paper/{}_cmip.zarr".format(
                variable
            )
        )
        .assign_coords({"year": np.arange(1975, 2100, 10)})
        .sel(year=slice(2080, 2099))
        .mean(dim="year")
        .probability.sel(scenario="ssp370")
        .mean(dim="gcm")
        .drop("scenario")
    )

In [ ]:
gcms = [
    ("MRI-ESM2-0", (0, 0)),
    ("MIROC-ES2L", (1, 0)),
    ("MPI-ESM1-2-LR", (2, 0)),
    ("ACCESS-ESM1-5", (3, 0)),
    ("ACCESS-CM2", (4, 0)),
    ("CanESM5-CanOE", (5, 0)),
]
titles = [
    "Burn area\n[%/year]",
    "Drought mortality\n[]",
    "Biotic agent mortality\n[]",
]

### Load in biomass data from National biomass and carbon database


In [ ]:
biomass = xr.open_rasterio(
    "https://carbonplan.blob.core.windows.net/carbonplan-data/raw/nbcd/NBCD_countrywide_biomass_mosaic.tif"
)

In [ ]:
reprojected_biomass = biomass.rio.reproject(plot.cartopy_proj_albers())

In [ ]:
# convert from tons per pixel to tons/hectare by dividing by 5.76 (the # hectares in a 240 m pixel) per the readme
maps["Biomass"] = reprojected_biomass / 5.76
# aggregate 64x64 240m cells for plotting
maps["Biomass"] = (
    maps["Biomass"]
    .coarsen(y=64, x=64, boundary="trim")
    .mean()
    .isel(band=0)
    .drop(["band", "spatial_ref"])
)

In [ ]:
# for plotting purposes mask out anywhere that is less than 1 ton/ha
maps["Biomass"] = maps["Biomass"].where(maps["Biomass"] > 1)

In [ ]:
plot_params = {
    "Fire": {
        "cmap": get_colormap("reds"),
        "var_lims": (0, 3),
        "label": "Burn area\n(%/year)",
        "panel": "A",
        "cbar_ylocation": 0.55,
        "cbar_xlocation": 0.44,
        "panel_location": (0, 0),
    },
    "Insects": {
        "cmap": get_colormap("blues"),
        "var_lims": (0, 0.8),
        "label": "Insect-related\nmortality (%/year)",
        "panel": "B",
        "cbar_ylocation": 0.55,
        "cbar_xlocation": 0.96,
        "panel_location": (0, 1),
    },
    "Drought": {
        "cmap": get_colormap("pinks"),
        "var_lims": (0, 4),
        "label": "Drought-related\nmortality (%/year)",
        "panel": "C",
        "cbar_ylocation": 0.05,
        "cbar_xlocation": 0.44,
        "panel_location": (1, 0),
    },
    "Biomass": {
        "cmap": get_colormap("greens"),
        "var_lims": (0, 300),
        "label": "Biomass (tons/ha)",
        "panel": "D",
        "cbar_ylocation": 0.05,
        "cbar_xlocation": 0.96,
        "panel_location": (1, 1),
    },
}

In [ ]:
def get_colormap(name):
    if name == "blues":
        return get_continuous_cmap(["#CFE0F9", "#588EF9", "#0432A5"])
    elif name == "pinks":
        return get_continuous_cmap(["#F9C7ED", "#E563BA", "#770361"])
    elif name == "reds":
        return get_continuous_cmap(["#F9D3BD", "#E87A3D", "#752003"])
    elif name == "greens":
        return get_continuous_cmap(["#C9E4BF", "#76BA74", "#36763C"])

In [ ]:
matplotlib.rc("font", family="sans-serif")
matplotlib.rc("font", serif="Helvetica Neue")
matplotlib.rc("text", usetex="false")
matplotlib.rcParams.update({"font.size": 14, "svg.fonttype": "none"})

In [ ]:
state_borders, us_border = plot.cartopy_borders()
fig, axarr = plt.subplots(
    nrows=2,
    ncols=2,
    figsize=(10, 6),
    subplot_kw={"projection": plot.cartopy_proj_albers()},
)
from mpl_toolkits.axes_grid1 import make_axes_locatable

for row, variable in enumerate(["Fire", "Drought", "Insects", "Biomass"]):
    vmin, vmax = (
        plot_params[variable]["var_lims"][0],
        plot_params[variable]["var_lims"][1],
    )
    map_plot = maps[variable].plot.imshow(
        ax=axarr[plot_params[variable]["panel_location"]],
        cmap=plot_params[variable]["cmap"],
        vmin=vmin,
        vmax=vmax,
        add_colorbar=False,
    )
    plot.map_pretty(axarr[plot_params[variable]["panel_location"]], title="")

    axarr[plot_params[variable]["panel_location"]].text(
        0.12,
        1.05,
        plot_params[variable]["panel"],
        transform=axarr[plot_params[variable]["panel_location"]].transAxes,
        fontsize=18,
    )
    cax = fig.add_axes(
        [
            plot_params[variable]["cbar_xlocation"],
            plot_params[variable]["cbar_ylocation"],
            0.018,
            0.14,
        ]
    )
    cbar = fig.colorbar(map_plot, cax=cax, orientation="vertical")

    cax.text(
        0.5,
        -0.22,
        plot_params[variable]["var_lims"][0],
        transform=cax.transAxes,
        horizontalalignment="center",
    )
    cax.text(
        0.5,
        1.05,
        plot_params[variable]["var_lims"][1],
        transform=cax.transAxes,
        horizontalalignment="center",
    )
    cax.text(
        1.8,
        0.5,
        plot_params[variable]["label"],
        transform=cax.transAxes,
        verticalalignment="center",
        multialignment="center",
        rotation=-90,
    )
    print(variable)
    cbar = fig.colorbar(map_plot, cax=cax, orientation="vertical")
    cbar.outline.set_visible(False)
    cbar.set_ticks([])
plt.tight_layout(pad=-2)
for format_string in ["svg", "png"]:
    fig.savefig(
        "Figure-Supp8." + format_string,
        format=format_string,
        bbox_inches="tight",
    )